In [23]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [16]:
# Create profanity list
bad_words = ["shit", "fuck", "damn", "bitch", "asshole", "ass", "dick", "pussy", "cock", "bastard", "douche"]

# Read in csv for movie database
movie_df = pd.read_csv("/Users/vaishnaviiparamashivam/Documents/Badwords_corr_proj/output/cleaned_movie_list.csv")

# Replace space in movie_links with '-'and remove '/Movie-Scripts' and 'Script'
movie_df['Movie_Links'].replace('\s+', '-',regex=True,inplace=True)
movie_df['Movie_Links'].replace('/Movie-Scripts/', '/',regex=True,inplace=True)
movie_df['Movie_Links'].replace('-Script', '',regex=True,inplace=True)
movie_df.head()

,Movie_Title,Movie_Links
0,10 Things I Hate About You,/10-Things-I-Hate-About-You.html
1,12,/12.html
2,12 and Holding,/12-and-Holding.html
3,12 Monkeys,/12-Monkeys.html
4,12 Years a Slave,/12-Years-a-Slave.html


In [17]:
# Create columns to store word counts
for word in bad_words:
    movie_df[word] = ""

movie_df["total_words"] = ""
movie_df["total_bad_words"] = ""
movie_df

,Movie_Title,Movie_Links,shit,fuck,damn,bitch,asshole,ass,dick,pussy,cock,bastard,douche,total_words,total_bad_words
0,10 Things I Hate About You,/10-Things-I-Hate-About-You.html,,,,,,,,,,,,,
1,12,/12.html,,,,,,,,,,,,,
2,12 and Holding,/12-and-Holding.html,,,,,,,,,,,,,
3,12 Monkeys,/12-Monkeys.html,,,,,,,,,,,,,
4,12 Years a Slave,/12-Years-a-Slave.html,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,You've Got Mail,/You've-Got-Mail.html,,,,,,,,,,,,,
1206,Youth in Revolt,/Youth-in-Revolt.html,,,,,,,,,,,,,
1207,Zero Dark Thirty,/Zero-Dark-Thirty.html,,,,,,,,,,,,,
1208,Zerophilia,/Zerophilia.html,,,,,,,,,,,,,


In [18]:
# Create a url list to loop through
movie_urls = movie_df["Movie_Links"].tolist()
movie_urls

['/10-Things-I-Hate-About-You.html',
 '/12.html',
 '/12-and-Holding.html',
 '/12-Monkeys.html',
 '/12-Years-a-Slave.html',
 '/127-Hours.html',
 '/1492:-Conquest-of-Paradise.html',
 '/15-Minutes.html',
 '/17-Again.html',
 '/187.html',
 '/2001:-A-Space-Odyssey.html',
 '/2012.html',
 '/25th-Hour.html',
 '/28-Days-Later.html',
 '/30-Minutes-or-Less.html',
 '/42.html',
 '/44-Inch-Chest.html',
 '/48-Hrs..html',
 '/50-50.html',
 '/500-Days-of-Summer.html',
 '/8-Mile.html',
 '/8MM.html',
 '/9.html',
 '/A-Few-Good-Men.html',
 '/A-Most-Violent-Year.html',
 '/A-Prayer-Before-Dawn.html',
 '/A-Quiet-Place.html',
 '/A-Scanner-Darkly.html',
 '/A-Serious-Man.html',
 '/A.I..html',
 '/Above-the-Law.html',
 '/Absolute-Power.html',
 '/Abyss,-The.html',
 '/Ace-Ventura:-Pet-Detective.html',
 '/Adaptation.html',
 '/Addams-Family,-The.html',
 '/Adjustment-Bureau,-The.html',
 '/Adventures-of-Buckaroo-Banzai-Across-the-Eighth-Dimension,-The.html',
 '/Affliction.html',
 '/After-School-Special.html',
 '/After.Lif

In [25]:
# Loop through urls, scrape html, get total/key word counts, add them to the dataframe
# This code will take a while to complete

for movie in movie_urls:
    try:
        url = 'https://www.imsdb.com/scripts'+ movie
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, "html5lib")
        pre = soup.pre
        script = pre.get_text()
        tokens = re.findall('\w+', script)
        words = []
        for word in tokens:
            words.append(word.lower())
        movie_df.loc[movie_df["Movie_Links"] == movie, "total_words"]  = len(words)
        total_count = 0
        for bword in bad_words:
            movie_df.loc[movie_df["Movie_Links"] == movie, bword] = words.count(bword)
    except:
        pass

In [26]:
# Display dataframe to confirm word counts
movie_df

,Movie_Title,Movie_Links,shit,fuck,damn,bitch,asshole,ass,dick,pussy,cock,bastard,douche,total_words,total_bad_words
0,10 Things I Hate About You,/10-Things-I-Hate-About-You.html,8,2,3,5,3,7,2,1,0,0,0,19152,
1,12,/12.html,1,0,1,0,1,0,0,1,0,2,0,14144,
2,12 and Holding,/12-and-Holding.html,10,4,3,1,5,4,0,7,0,1,0,18999,
3,12 Monkeys,/12-Monkeys.html,4,5,2,2,1,3,0,1,0,2,0,32272,
4,12 Years a Slave,/12-Years-a-Slave.html,0,0,12,1,0,0,0,0,0,3,0,31014,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,You've Got Mail,/You've-Got-Mail.html,5,4,1,1,1,1,1,0,0,0,0,25300,
1206,Youth in Revolt,/Youth-in-Revolt.html,5,3,5,0,4,5,1,1,2,1,0,26709,
1207,Zero Dark Thirty,/Zero-Dark-Thirty.html,3,14,1,1,1,3,1,0,0,0,0,22200,
1208,Zerophilia,/Zerophilia.html,4,12,6,3,4,5,0,0,0,1,0,15061,


In [27]:
# Drop movies with 0 total words or blank values
movie_df = movie_df[movie_df.total_words !=0]
movie_df = movie_df[movie_df.total_words != '']
movie_df

,Movie_Title,Movie_Links,shit,fuck,damn,bitch,asshole,ass,dick,pussy,cock,bastard,douche,total_words,total_bad_words
0,10 Things I Hate About You,/10-Things-I-Hate-About-You.html,8,2,3,5,3,7,2,1,0,0,0,19152,
1,12,/12.html,1,0,1,0,1,0,0,1,0,2,0,14144,
2,12 and Holding,/12-and-Holding.html,10,4,3,1,5,4,0,7,0,1,0,18999,
3,12 Monkeys,/12-Monkeys.html,4,5,2,2,1,3,0,1,0,2,0,32272,
4,12 Years a Slave,/12-Years-a-Slave.html,0,0,12,1,0,0,0,0,0,3,0,31014,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,You've Got Mail,/You've-Got-Mail.html,5,4,1,1,1,1,1,0,0,0,0,25300,
1206,Youth in Revolt,/Youth-in-Revolt.html,5,3,5,0,4,5,1,1,2,1,0,26709,
1207,Zero Dark Thirty,/Zero-Dark-Thirty.html,3,14,1,1,1,3,1,0,0,0,0,22200,
1208,Zerophilia,/Zerophilia.html,4,12,6,3,4,5,0,0,0,1,0,15061,


In [29]:
# Export DataFrame to CSV
movie_df.to_csv('/Users/vaishnaviiparamashivam/Documents/Badwords_corr_proj/output/movie_bword_counts.csv')